# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc

In [3]:
#Search parameters
y1 = '2021'
#y0 = '2013'

sample = 'acs1'

phx_pumas = ['0400112','0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP'

In [4]:
#AMI bands in 2021 by household size

AMI_50pct = {'1':27650,'2':31600,'3':35500,'4':39500,'5':42700,'6':45850,\
            '7':49000,'8':52150}
AMI_80pct = {'1':44250,'2':50600,'3':56900,'4':63200,'5':68300,'6':73350,\
            '7':78400,'8':83450}
AMI_100pct = {'1':55300,'2':63200,'3':71100,'4':79000,'5':85400,'6':91700,\
            '7':98000,'8':104300}

In [5]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [6]:
raw = get.get_puma(sample,y1,data_cols)

In [7]:
df = raw.copy()

In [8]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(int)

In [9]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],\
                   labels=['1','2','3','4','5','6','7','8'])

In [10]:
df.head()

,GEO_ID,HINCP,NP,WGTP,WGTP1,WGTP2,WGTP3,WGTP4,WGTP5,WGTP6,...,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80,HHSz
4,0400119,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
33,0400129,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
38,0400120,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
40,0400116,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
46,0400129,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
dff = df[~(df.HHSz.isna()) & (df.HINCP!=-60000)].copy()
print(dff.WGTP.sum())
dff['HHSz'] = dff['HHSz'].astype(str)

666238


In [12]:
#assign AMI range based on household size and ami dictionaries
dff['AMI_range'] = np.where((dff['HINCP']<=dff['HHSz'].map(AMI_50pct)),'u50_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_50pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_80pct)),\
                   '50_80_ami',np.where((dff['HINCP']>dff['HHSz'].map(AMI_80pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_100pct)),\
                   '80_100_ami','o100_ami')))

In [13]:
pivot = dff.drop(['HINCP','NP','HHSz'],axis=1).copy()
pivot = pivot.groupby(['GEO_ID','AMI_range']).sum().reset_index()
pivot.rename(columns={'WGTP':'hh_e'},inplace=True)

In [14]:
pivot['hh_SE'] = pivot.apply(lambda x: (calc.get_se(x['hh_e'],x[repwts])),axis=1)
pivot['hh_MOE'] = pivot.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
pivot['hh_CV'] = pivot.apply(lambda x: (calc.get_cv(x['hh_e'],x['hh_SE'])),axis=1)
pivot = pivot.drop(columns=repwts)

In [15]:
final = pd.pivot_table(pivot,values=['hh_e','hh_MOE','hh_CV'],index='GEO_ID',\
                      columns=['AMI_range'],aggfunc=np.sum).reset_index()

In [16]:
final

GEO_ID     hh_CV                                      hh_MOE  \
AMI_range          50_80_ami 80_100_ami  o100_ami   u50_ami    50_80_ami   
0          0400112  7.553053   7.326393  1.850860  5.982674  1077.735343   
1          0400113  6.930928   7.840724  2.457191  5.967345   957.268666   
2          0400114  5.549114   6.293865  3.030888  4.899772  1270.957809   
3          0400115  5.440686   7.600760  3.491819  4.845096  1538.809511   
4          0400116  5.077273   5.385707  2.700971  3.812287  1279.671764   
5          0400117  6.530118   7.637212  2.676352  5.135517  1467.018341   
6          0400118  5.625607   6.089197  2.830769  4.182366  1278.278102   
7          0400119  5.137363   8.788017  3.504936  4.132723   900.143752   
8          0400120  7.768447   9.625971  2.165595  6.750508  1114.565893   
9          0400121  5.662385   8.321738  3.221893  4.624055  1550.948974   
10         0400122  5.649761   7.251448  4.924724  4.889995  1164.211140   
11         0400123  5.369766   8.317253  4.253306  5.417595  1462.371669   
12         0400125  5.566549   9.905888  4.005972  4.774746  1373.313918   
13         0400128  7.777601   7.429192  2.739979  6.951083   861.849395   
14         0400129  7.581883   7.387722  1.992801  6.570279  1218.695810   

                                                      hh_e             \
AMI_range   80_100_ami     o100_ami      u50_ami 50_80_ami 80_100_ami   
0           666.331815  2150.989523  1080.795358      5273       3361   
1           744.299242  1826.275099  1139.064687      5104       3508   
2           865.022511  1885.151501  1466.965424      8464       5079   
3           874.544877  1617.282254  1605.042694     10452       4252   
4           871.953854  1678.414709  1280.645944      9314       5983   
5           965.124949  1921.955340  1182.481879      8302       4670   
6          1051.924766  1431.218753  1803.344314      8397       6384   
7           788.088881  1454.439404  1152.881441      6475       3314   
8           787.695071  1682.684865  1064.420604      5302       3024   
9          1228.627446  1531.495795  1329.986493     10122       5456   
10          724.662644  1272.007299  1400.522390      7615       3693   
11         1023.379442  1174.549727  1515.566006     10064       4547   
12         1011.642601  1758.507000  1559.385188      9117       3774   
13          844.149345  1960.381189  1493.874600      4095       4199   
14          649.119480  2038.714691   993.331653      5940       3247   

                            
AMI_range o100_ami u50_ami  
0            42947    6676  
1            27466    7054  
2            22985   11064  
3            17116   12242  
4            22964   12414  
5            26538    8509  
6            18684   15934  
7            15335   10309  
8            28714    5827  
9            17566   10629  
10            9545   10584  
11           10205   10338  
12           16222   12069  
13           26440    7942  
14           37806    5587

In [17]:
final.to_excel('output/hhinc_ami_pumas_21.xlsx')

### PUMA - median HHI by HH Size, total HHs by Size

pum_size = get.get_puma('acs/acs1',y1,cols_inc_size)
pum_size = clean_table(pum_size)
pum_size = pum_size[pum_size['GEO_ID'].isin(phx_pumas)]

pumas_rename = {'B19019_001E':'hh_tot_E','B19019_001M':'hh_tot_M','B19019_002E':'hh_1_E','B19019_002M':'hh_1_M',\
               'B19019_003E':'hh_2_E','B19019_003M':'hh_2_M','B19019_004E':'hh_3_E','B19019_004M':'hh_3_M',\
               'B19019_005E':'hh_4_E','B19019_005M':'hh_4_M','B19019_006E':'hh_5_E','B19019_006M':'hh_5_M',\
               'B19019_007E':'hh_6_E','B19019_007M':'hh_6_M','B19019_008E':'hh_7_E','B19019_008M':'hh_7_M'}

pum_size.rename(columns=pumas_rename,inplace=True) 

pum_size = geo_agg.sumgeo_cv(pum_size,'GEO_ID')

pum_size

pum_size.to_excel('output/medhhi_hhsize.xlsx',index=False)